In [42]:
import polars as pl
import numpy as np
from datetime import datetime,timedelta
    from scipy.stats import skew, kurtosis

In [31]:
data = pl.read_parquet('test.parquet')
data

code,date,open,high,low,close,volume
i32,date,f64,f64,f64,f64,i32
139,2005-01-03,196.303346,201.579617,193.957671,198.052344,4845545
139,2005-01-04,201.291188,203.907249,196.85356,201.024511,1665324
139,2005-01-05,203.822877,204.5694,199.43191,202.489505,5144678
139,2005-01-06,205.577968,209.473182,199.693323,205.407583,1257428
139,2005-01-10,197.220564,202.567173,192.748399,196.875319,3472705
…,…,…,…,…,…,…
98380,2025-05-20,24230.491595,24763.152159,23843.208541,24374.28064,6394214
98380,2025-05-21,24778.660901,25180.883122,24622.180278,24885.010751,4265126
98380,2025-05-22,25623.015354,26097.546318,25167.413061,25823.065523,1353757


In [33]:
factor = pl.read_parquet('factor.parquet')
factor_name = 'factor_name'
factor

code,date,factor_name
i32,date,f64
139,2005-01-03,0.061016
139,2005-01-04,0.297761
139,2005-01-05,0.608341
139,2005-01-06,0.037301
139,2005-01-10,0.533844
…,…,…
98380,2025-05-20,0.717559
98380,2025-05-21,0.939378
98380,2025-05-22,0.535352


In [45]:
#第一步：排序因子值然后计算因子的波动方差分布等统计指标
factor_ = factor.clone().sort(['code','date'])
#总样本的指标
values = factor_.select(pl.col(factor_name)).to_series()
total = pl.DataFrame({
    "stat": ["factor_name", "mean", "var (ddof=1)", "std (ddof=1)",'information ratio','skewness','kurtosis(excess)'    ],
    "value": [
        factor_name,
        values.mean(),
        values.var(ddof=1),
        values.std(ddof=1),
        values.mean()/values.std(ddof=1),
        values.skew(),
        values.kurtosis()
        
    ]
},strict=False)


In [46]:
total

stat,value
str,str
"""factor_name""","""factor_name"""
"""mean""","""0.500449"""
"""var (ddof=1)""","""0.083189"""
"""std (ddof=1)""","""0.288426"""
"""information ratio""","""1.735104"""
"""skewness""","""-0.002468"""
"""kurtosis(excess)""","""-1.200246"""
